In [1]:
import gender_guesser.detector as gender
from tqdm.notebook import tqdm
from time import sleep

import os
import math
import warnings

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

import requests
from bs4 import BeautifulSoup 
import re

from sklearn.model_selection import cross_val_score, train_test_split,cross_validate
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [2]:
df00 = pd.read_csv("data/dataset-of-00s.csv")
#df00.shape
df10 = pd.read_csv("data/dataset-of-10s.csv")
df60 = pd.read_csv("data/dataset-of-60s.csv")
df70 = pd.read_csv("data/dataset-of-70s.csv")
df80 = pd.read_csv("data/dataset-of-80s.csv")
df90 = pd.read_csv("data/dataset-of-90s.csv")

In [3]:
def scrape(url):
    url_list = []
    r = requests.get(url)
    s = BeautifulSoup(r.text, "html.parser")
    for i in s.find_all("a"):
        href = i.attrs['href']
        if href.startswith("/"):
            site = url + href
            if site not in url_list:
                url_list.append(site)
                #print(site)
    return url_list
    

In [4]:
gender_info = {'F': ['she','her'],
               'M': ['he', 'him', 'his'],
               'Band': ['they', 'them', 'their', 'band']}

def count_pronouns(bio):
    pronoun_counts = [len(re.findall(rf"\W({'|'.join(pns)})\W", bio)) for pns in gender_info.values()]
    return pronoun_counts


def get_gender_info(artist_url):
    
    my_page = requests.get(artist_url)
    soup = BeautifulSoup(my_page.content,"html.parser")
    #print(soup)

    About_tag = soup.find("h2",string="About")

    About_section = About_tag.find_parent()

    text = About_section.text
    print(count_pronouns(text))
    
    info = list(gender_info.keys())[np.argmax(count_pronouns(text))]
    
    return info
    

list(gender_info.keys())[2]

'Band'

In [59]:

artist_url_dict = {}
word = 'artist'
artist_urls_00 = []
song_url_list = []

for n in df00['uri']:
    track = n.split(":")[-1]
    song_url = 'https://open.spotify.com/track/' + track
    song_url_list.append(song_url)

print(len(song_url_list))
for song in tqdm(range(len(song_url_list))):
    url_lst = scrape(song_url_list[song])
    count = 0
    
    for u in range(len(url_lst)):
        if word in url_lst[u]:
            artist_urls_00.append(url_lst[u])
                 
    #print(len(artist_urls_00))  
    #print(len(song_url_list[0:song])+1)
    
    #print(len(song_url_list[0:song]))
    if (len(artist_urls_00)) != len(song_url_list[0:song])+1:
        #print("Yes")
        
        artist_urls_00.append(song_url_list[song]) 
                 
            
        # else:
        #     artist_urls_00.append(url_lst[-1])
        #     break
    

            #print(u)
#         else:
#             count += 1
#             artist_urls_00.append(url_lst[1])
    
print(len(artist_urls_00))

artist_url_dict = {}
artist_urls_final00 = []
for n in range(len(artist_urls_00)):
    artist_id = artist_urls_00[n].split("/")[-1]
    artist_url = "https://open.spotify.com/artist/" + artist_id
    #print(artist_url)
    artist_urls_final00.append(artist_url)
    #artist_url_dict[df00['artist'][n]] = artist_url
    
    
print(len(artist_urls_final00))    
    
np.save('artist_urls_final00',artist_urls_final00) 
print(count,len(artist_urls_final00),df00.shape,artist_urls_final00[0:15])

5872


  0%|          | 0/5872 [00:00<?, ?it/s]

5872
5872
0 5872 (5872, 19) ['https://open.spotify.com/artist/3bGXaFVQLASmDMdjjeJr8a', 'https://open.spotify.com/artist/6XwcepfAsPhrvwziGxhwju', 'https://open.spotify.com/artist/7zDtfSB0AOZWhpuAHZIOw5', 'https://open.spotify.com/artist/4jGKpLbPpdhjmXkJSXbqtK', 'https://open.spotify.com/artist/1By9QBFnjZAoI83BZppHlt', 'https://open.spotify.com/artist/296MWsDlkDsnLb043EeIF8', 'https://open.spotify.com/artist/5Rk9nXVSA2RzBdef3aTJPR', 'https://open.spotify.com/artist/4mxWe1mtYIYfP040G38yvS', 'https://open.spotify.com/artist/6roFdX1y5BYSbp60OTJWMd', 'https://open.spotify.com/artist/3ImqpXKKgkH5VydfgX1hd4', 'https://open.spotify.com/artist/01U4snBkRWq0FBrK3P1mwM', 'https://open.spotify.com/artist/3FfvYsEGaIb52QPXhg4DcH', 'https://open.spotify.com/artist/209McJAEbNbvQXsnuNJZJw', 'https://open.spotify.com/artist/1XhhO2dTs2fKyhILj0aam6', 'https://open.spotify.com/artist/6Dy7t6ELZYEMhw9WNZ9IIF']


In [62]:
#get_gender_info(artist_urls_final[0])
artist_urls_final00 = list(np.load('artist_urls_final00.npy'))

gender_info_list00 = []
for url in tqdm(artist_urls_final00):
    my_page = requests.get(url)
    soup = BeautifulSoup(my_page.content,"html.parser")
    #print(soup)

    About_tag = soup.find("h2",string="About")
    try:
        About_section = About_tag.find_parent()
        count = count_pronouns(About_section.text)
        if sum(count) != 0:
            idx = np.argmax(count)
            info = list(gender_info.keys())[idx]
            #print(idx)
            gender_info_list00.append(info)
            
        else:
            gender_info_list00.append("unknown")
        
        
    except:
        #print(url)
        gender_info_list00.append("unknown")
        #print("exception!")
        
    
    
    
np.save('gender_info_list00',gender_info_list00)     
print(len(gender_info_list00),gender_info_list00[0:10])   

  0%|          | 0/5872 [00:00<?, ?it/s]

5872 ['Band', 'unknown', 'unknown', 'unknown', 'M', 'unknown', 'Band', 'M', 'M', 'Band']


## Final DataFrames saved to CSV

In [15]:
import gender_guesser.detector as gender

gender_info_list00 = list(np.load('gender_info_list00.npy'))
d = gender.Detector()

for n in range(len(gender_info_list00)):
    if gender_info_list00[n] == "unknown":
        first_name = (df00['artist'][n]).split(' ')[0]
        guessed = d.get_gender(first_name)
        if guessed == 'female' or guessed == 'mostly_female':
            gender_info_list00[n] = 'F'
        elif guessed == 'male' or guessed == 'mostly_male':
            gender_info_list00[n] = 'M'
        else:
            pass
            
            
print(df00.shape,len(gender_info_list00))  
print(gender_info_list00[0:20])

#np.unique(gender_info_list00)

index = 2


#df00.insert(loc=index, column='artist_gender', value=gender_info_list00)
df00.head(15)

df00.to_csv('df00.csv')

(5872, 20) 5872
['Band', 'unknown', 'unknown', 'unknown', 'M', 'unknown', 'Band', 'M', 'M', 'Band', 'unknown', 'M', 'Band', 'M', 'M', 'unknown', 'Band', 'Band', 'M', 'Band']


In [18]:
gender_info_list10 = list(np.load('gender_info_list10.npy'))

d = gender.Detector()

for n in range(len(gender_info_list10)):
    if gender_info_list10[n] == "unknown":
        first_name = (df10['artist'][n]).split(' ')[0]
        guessed = d.get_gender(first_name)
        if guessed == 'female' or guessed == 'mostly_female':
            gender_info_list10[n] = 'F'
        elif guessed == 'male' or guessed == 'mostly_male':
            gender_info_list10[n] = 'M'
        else:
            pass
            
            
print(df10.shape,len(gender_info_list10))  
print(gender_info_list10[0:20])

#np.unique(gender_info_list00)

index = 2


#df10.insert(loc=index, column='artist_gender', value=gender_info_list10)
(df10.head(15))

#df10.to_csv('df10.csv')

(6398, 20) 6398
['F', 'M', 'M', 'Band', 'M', 'F', 'Band', 'M', 'unknown', 'M', 'M', 'M', 'M', 'Band', 'M', 'M', 'unknown', 'M', 'M', 'M']


,track,artist,artist_gender,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,chorus_hit,sections,target
0,Wild Things,Alessia Cara,F,spotify:track:2ZyuwVvV6Z3XJaXIFbspeE,0.741,0.6260,1,-4.826,0,0.0886,0.02000,0.00000,0.0828,0.7060,108.029,188493,4,41.18681,10,1
1,Surfboard,Esquivel!,M,spotify:track:61APOtq25SCMuK0V5w2Kgp,0.447,0.2470,5,-14.661,0,0.0346,0.87100,0.81400,0.0946,0.2500,155.489,176880,3,33.18083,9,0
2,Love Someone,Lukas Graham,M,spotify:track:2JqnpexlO9dmvjUMCaLCLJ,0.550,0.4150,9,-6.557,0,0.0520,0.16100,0.00000,0.1080,0.2740,172.065,205463,4,44.89147,9,1
3,Music To My Ears (feat. Tory Lanez),Keys N Krates,Band,spotify:track:0cjfLhk8WJ3etPTCseKXtk,0.502,0.6480,0,-5.698,0,0.0527,0.00513,0.00000,0.2040,0.2910,91.837,193043,4,29.52521,7,0
4,Juju On That Beat (TZ Anthem),Zay Hilfigerrr & Zayion McCall,M,spotify:track:1lItf5ZXJc1by9SbPeljFd,0.807,0.8870,1,-3.892,1,0.2750,0.00381,0.00000,0.3910,0.7800,160.517,144244,4,24.99199,8,1
5,Here's To Never Growing Up,Avril Lavigne,F,spotify:track:0qwcGscxUHGZTgq0zcaqk1,0.482,0.8730,0,-3.145,1,0.0853,0.01110,0.00000,0.4090,0.7370,165.084,214320,4,32.17301,12,1
6,Sex Metal Barbie,In This Moment,Band,spotify:track:75BGF4LC7AOLFfxn6ukZDH,0.533,0.9350,0,-3.704,1,0.1280,0.01390,0.00000,0.1680,0.4810,140.092,262493,4,21.04510,14,0
7,Helluva Night,Ludacris,M,spotify:track:0flKDWZq11997Fb2ptkQvu,0.736,0.5220,2,-8.020,1,0.1160,0.02990,0.00000,0.1080,0.3690,97.547,200387,4,60.21027,10,1
8,Holiday With HH,No Bros,unknown,spotify:track:7LBa0KNFR8IY3g7LOfXqu8,0.166,0.9850,7,-2.886,1,0.1700,0.00183,0.01420,0.9580,0.1390,174.725,252787,4,31.23583,11,0
9,My Last,Big Sean Featuring Chris Brown,M,spotify:track:70tFuqBcduJv15bEnOPRTh,0.387,0.7730,8,-5.685,1,0.1700,0.09800,0.00000,0.2090,0.3680,78.629,254120,4,23.30245,9,1
